# Stock Recommender System

In this notebook, we will create the basic stock recommender system, using knowledge-based approaches, collaborative filtering and ranking (based on stock predictions).

First we load all relevant items:

In [2]:
import pandas as pd
import numpy as np
import sys, os
import sys
sys.path.insert(1, '..')
import recommender as rcmd
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn_recommender as rec
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer

Next we will load and pre-process the relevant data

In [ ]:
# retrieve all relevant symbols
stocks = fmp.profile.list_symbols()
cache = rcmd.stocks.Cache()

# load the relevant profile informations
df_profile = cache.load_profile_data()

# generate glove embeddings
rec.glove.download('twitter')
gt = rec.glove.GloVeTransformer('twitter', 25, 'sent', tokenizer=rec.nlp.tokenize_clean)
embs = gt.transform(df_profile['description'].fillna(""))
df_embs = pd.concat([df_profile[['symbol']], pd.DataFrame(embs)], axis=1).set_index('symbol')

# create dummy for categorical values
df_sector_dummy = pd.get_dummies(df_profile['sector'], dummy_na=True, prefix='sector')
df_industry_dummy = pd.get_dummies(df_profile['industry'], dummy_na=True, prefix='industry')
df_exchange_dummy = pd.get_dummies(df_profile['exchange'], dummy_na=True, prefix='exchange')
df_cats = pd.concat([df_profile[['symbol']], df_sector_dummy, df_industry_dummy, df_exchange_dummy], axis=1).set_index('symbol')